# Hands-on Lab 8: Controls Part 2

### EECS 16B: Designing Information Devices and Systems II, Spring 2022

Written by Vivek Athalye and Andrew Blatner (2017). Revised by Peter Schafhalter (2019).

Updated by Megan Zeng, Steven Lu (2022)

## Table of Contents

* [Controls Part 1 Lab Note](https://www.eecs16b.org/labs/notes/handson_note7_controls1_sp22.pdf)
* [Introduction](#intro)
* [Part 1: Analyzing a Turn](#part1)
* [Part 2: Implementing Turns](#part2)
* [Part 3: Correcting Mechanical Errors](#part3)
* [Part 4: Micboard Verification](#part4)

<a id='intro'></a>
## Introduction

In the previous lab, we developed a closed-loop controller to make the car go straight. However, we also want the car to be able to turn left and right! In this lab, we will modify the control scheme from Lab 7 to enable the car to turn. For a refresher on our closed-loop controller, please refer to the Controls Part 1 lab note [here](https://www.eecs16b.org/labs/notes/handson_note7_controls1_sp22.pdf). There is no lab note for Lab 8 and everything you need is in this notebook!

## Optional Pre-Lab

### Complete the optional pre-lab assignment before doing the lab. For all students, submit your answers to the Gradescope assignment "[OPTIONAL] [ALL LAB] Pre-Lab 8: Controls Part 2". Please complete Pre-Lab 8 by Sunday, April 3 at 11:59pm.


### Help Request Link: https://links.eecs16b.org/lab-help-sp22

<a id='part0'></a>
## <span style="color:red">Part 0: Obligatory Launchpad Warning</span>

### Remember to never power your Launchpad with both the USB and the 5V jumper. To be safe, always unplug your batteries to power down your breadboard before interfacing your Launchpad with the computer.

<a id='part1'></a>
## <span style="color:navy">Part 1: Analyzing a Turn</span>

### Materials
- S1XT33N Car
- Closed-loop control scheme

First, verify that your closed-loop control scheme is still functional. Upload `closed_loop.ino` onto your Launchpad and make sure your car drives straight.

### We want to turn with a specific radius.  Which open-loop control input would lead to turning?  
During open-loop control, we sought to drive the car straight at our desired velocity $v^*$.  

We used our model of the car ($\theta$, $\beta$) to calculate the open-loop control input $u^{OL}_{L,R}$ which drives the car straight in the absence of perturbations:  

$$u^{OL}_L = \frac{v^*+\beta_L}{\theta_L} $$
$$u^{OL}_R = \frac{v^*+\beta_R}{\theta_R} $$

Is it possible to calculate an open-loop control input which would result in turning with a desired radius? While you may have seen the car turn while running `open_loop.ino` from Controls or while collecting data in System ID, just like open-loop for driving straight, open-loop for turning is also subject to noise, perturbations, and model mismatch that make it difficult to maintain a steady turn with a fixed radius. To make our turns more reliable, we will instead *turn* to closed-loop control.


### We have been turning during Closed-Loop Control!
We can get a clue about turning by considering how we implemented closed-loop control. We perform feedback control of S1XT33N in order to minimize the difference between the two wheels ($\delta[i] = d_L[i] - d_R[i]$). When perturbations cause one wheel to get ahead of the other resulting in non-zero $\delta$, feedback control turns the car to correct the error.

Let's walk through this.  

Our feedback control policy is: 
$$u_L[i] = u_L^{OL} - \frac{f_L}{\theta_L}\delta[i]$$

$$u_R[i] = u_R^{OL} + \frac{f_R}{\theta_R}\delta[i]$$


Let's say the right wheel has moved further than the left ($d_R[i] > d_L[i]$), resulting in a negative $\delta[i]$. A negative $\delta[i]$ results in a reduction of $u_R[i]$, and an increase of $u_L[i]$. But what does this mean physically?

If the right wheel is getting ahead of the left one, the left wheel will need to move a little faster and the right wheel a little slower for the wheels to even back out. Thus, feedback control corrects errors by turning to compensate for them.  

So $\delta \ne 0$ can be used to turn the car. Let's get more specific.  


### Turning via reference tracking

We would like the car to turn with a specified radius $r$ and linear velocity $v^*$. The controller's unit for distance is encoder ticks, but each tick is approximately 1 cm of wheel circumference.

To turn, we need to manually add some $\delta_{ref}$ to our $\delta$ value in every time step during the turn. By doing so, we are tricking our control scheme into thinking that our car has turned by some amount and generated that extra $\delta_{ref}$ of error, and cause it to turn in the other direction to compensate. Without loss of generality, we'll analyze a right turn, corresponding to adding a **negative** $\delta_{ref}$ value (we trick the car into thinking it has turned left, so it turns right as "compensation"). For a left turn, we simply add in a **positive** $\delta_{ref}$ instead. Our goal is to generate this $\delta_{ref}$ from the desired $r$ and $v^*$ for the controller to follow. It will be a function of the controller's time-step, as we want our car to turn gradually, rather than pivot turn.

We define the following variables:
- $i$ [time] - timestep
- $r$ [cm] - turn radius of the center of the car; 1 cm ~ 1 encoder tick
- $d$ [tick] - distance traveled by the center of the car
- $l$ [cm] - distance between the centers of the wheels; 1 cm ~ 1 encoder tick
- $\omega$ [rad/time] - angular velocity
- $\theta$ [rad] - angle turned

Inspect the following diagram:

<center>
<img width="400px" src="images/turning_figure.png">
</center>



From this geometry, can you write $\delta_{ref}[i]$ in the following form?
$$\delta_{ref}[i] = f(r,v^*,l,i)$$

Hint 1: Arc length = radius * central angle
<br>
Hint 2: Distance travelled = velocity * time


<h3><span style='color:red'>**Check your answer with the lab staff!**</span></h3>

**Answer:**

1. $v_{star}*i = \omega *r*i = r*\theta[i]$
2. $\theta[i] = \frac{v_{star}*i}{r}$
3. $d_L = (r + \frac{l}{2})\theta$
4. $d_R = (r - \frac{l}{2})\theta$
5. $\delta_{ref}[i] = d_L - d_R$
6. $\delta_{ref}[i] = (r + \frac{l}{2} - r + \frac{l}{2})\theta$
7. $\delta_{ref}[i] = l*\theta$
8. $\delta_{ref}[i] = l*(\frac{v_{star}*i}{r})$


$\delta_{ref}[i] = \frac{v_{star} * l * i}{r}$

<a id='part2'></a>
## <span style="color:navy">Part 2: Implementing Turns</span>

### Materials
- S1XT33N Car
- Closed-loop control scheme


### Sampling periods 
 
In our implementation, the control loop and the data collection have different periods because we want the controller to update the trajectory every tenth of a second instead of every half a second, which was our data collection rate. Because of this, we run into an issue when we're trying to implement turning at the same speed by adding in some $\delta_{ref}$ in each time step, as the definition of the time step is dependent on the sampling period we use. To fix this, we need to change our v* when we use it while implementing turning, or else the car will not move at the correct velocity while it turns.

Let the sampling periods of the control loop and the data collection be $T_c$ and $T_d$, respectively, and their sampling frequencies $F_c$ and $F_d$, respectively.

$F_c$ is an integer multiple of $F_d$: $m=\frac{F_c}{F_d} \in \mathbb{Z}^{+}$

To ensure the car is travelling at the same ground velocity in both cases, we want to have a constant distance, measured in ticks, travelled per second, or $\frac{ticks}{s}$.

$F_c$, with units [$\frac{sample_c}{s}$], is the number of times the closed-loop is sampled every second, and $F_d$, with units [$\frac{sample_d}{s}$], is the number of times data collection was sampled per second. Let $v^*_c$ be the velocity of closed-loop, and $v^*$ be the velocity of the data collection. $v^*_c$ has the units of [$\frac{ticks}{sample_c}$] and $v^*$ has units [$\frac{ticks}{sample_d}$].

To get a constant $\frac{ticks}{s}$, we need $v^*_c * F_c = v^* * F_d$.

Solving for $v^*_c$, we get $v^*_c = v^* * \frac{F_d}{F_c} = \frac{v^*}{m}$

We use $T_c=100$ms, $T_d=500ms$, so $F_c=10$, $F_d=2$, so $m=5$.


**<span style='color:violet'>- Note: The controller updates every tenth of second (1/10 of a second is 100ms, thus $T_c = 100ms$). This means that it collects 10 samples every 1 second ($F_c = 10$).</span>**

**<span style='color:violet'>- Note: The data collector updates every half of a second (1/2 of a second is 500ms, thus $T_d = 500ms$). This means that it collects 2 samples every 1 second ($F_d = 2$).</span>**

As an example, during data collection, we sampled every .5s, so the velocity v* is defined with respect to that sampling period. If our v* was collected to be 100 ticks/sample, that means every .5 seconds, we expect our car to move 100 ticks (assuming constant velocity). Now, in our controller, our sampling rate is now .1 seconds. If we use the same v* of 100 ticks/sample, we are now saying that we expect our car to move 100 ticks in .1 seconds, because the controller does not have the information that the original sampling period during data collection was .5 seconds. That means the velocity has effectively quintupled! To account for this, we will divide v* by 5 when using a v* value for turning.

**<span style='color:red'> In your delta_reference function, use $\frac{v^*}{m}$ instead of $v^*$. Do not replace the original $v^*$ parameter defined in `CODE BLOCK CON1`!</span>**

### Implementation
- Open `turning.ino`.
- Copy the required code from `closed_loop.ino` into **`CODE BLOCK CON1/2/3`**
    - Make sure to update pin numbers if you've made changes.
    - <span style='color:red'>**Note that your `driveStraight` functions only have one parameter in this program unlike closed_loop.ino, so only copy over the body of the functions!**</span>
- **Complete the function `delta_reference` in `CODE BLOCK CON4`**.
    - Look at `CODE BLOCK CON0` to see how this is used!
    - **Use $\frac{v^*}{m}$ instead of $v^*$ in your delta_reference function!**
    - Use the variables `TURN_RADIUS`, `v_star`, `CAR_WIDTH`, and `i` (the time-step).
    - `delta_ss` is used in `CODE BLOCK CON0`, so don't include it here.
- The commands that your car follows is under **`PREPROGRAMMED PATH`**
    - `NUM_COMMANDS` is the number of commands your car will follow. The default is set as 4.
    - The commands your car follows as well as the run time associated with each command is in the `run_times` and `drive_modes` arrays.
        - The number of elements in both `drive_modes` and `run_times` is equal to `NUM_COMMANDS`
        - `drive_modes` can take on values: `[DRIVE_STRAIGHT, DRIVE_LEFT, DRIVE_RIGHT]`.
        - Try to tune your `run_time` and `turn_radius` for the turns such that the car turns approximately 90 degrees.
            - **The values in `run_times` are in ms**, so you should have run times that are at least 1000.
            - If your car doesn't turn with the same speed when it's turning left vs right, you can adjust the run times of the left and right turns to be different values.
        
**Remember**: If you have previously changed out your motor or encoder pins, you will need to update those pin assignments in `turning.ino` accordingly!

When you're done, try it out! S1XT33N should follow the preprogrammed path we provided as a default. Feel free to play around with implementing different paths by changing the commands under `PREPROGRAMMED PATH` if you would like.

<a id='part3'></a>
## <span style="color:navy">Part 3: Correcting Mechanical Errors</span>

At this stage, your car may already go perfectly straight and turn left and right equally. However, some cars may not go quite straight even when both encoder measurements are exactly the same ($\delta=constant$). This can be caused by mechanical issues such as axle wobble or mismatch in the wheel sizes. In this case, the car thinks it's driving straight, so it's the best we can do using just the sensors in our control system.

If the car is consistently turning slightly instead of driving straight, we can correct it by instructing the car to slightly turn in the opposite direction with a radius of `STRAIGHT_RADIUS`. Look at your `delta_reference()` function when deciding whether what you return is positive or negative.

**If you think your car could benefit from this, implement the function `straight_correction` in `CODE BLOCK CON5` using the variable `STRAIGHT_RADIUS`. Update this value from INFINITY to a suitably large number.**

<a id='part4'></a>
## <span style="color:navy">Part 4: Micboard Verification</span>
-----

We will be using the micboard next week for the SVD/PCA lab! Verify that your biasing circuits and front-end circuitry still work as expected.

### Materials:
- Oscilloscope
- Launchpad

### Tasks:
1. Your mic board VDD should be connected to the 5V rail, which is powered by the 5V voltage regulator.
2. Probe the output of the micboard with the oscilloscope.
3. Display a peak-to-peak measurement for the current channel on the oscilloscope.
    - Press "Meas", select Peak-to-Peak, and press "Add measurement"
4. Display an average measurement for the current channel on the oscilloscope.
    - Press "Meas", select Avg - FS, and press "Add measurement"
5. Tune your mic board until you see a signal with $\approx2.5\mathrm{V}_{pp}$ amplitude centered near $1.65\mathrm{V}$ while you speak from a comfortable distance away from the microphone.
    - Lower the gain: Turn the potentiometer on your mic board **clockwise**.
    - Increase the gain: Turn the gain potentiometer on the mic board **counter-clockwise**.
6. Now place your oscillocope cable at the output of the non-inverting amplifier that follows your low pass filter. Verify that the output of the amplifier in this circuit is working as expected when you speak.
7. Upload `adc_read.ino` to the Launchpad.
8. Connect `P6.0` to the output of the non-inverting amplifier that follows your low pass filter.
    - If `P6.0` no longer works, choose another analog read pin (`P6.0-6.6`, `P7.0`) to use and change all instances of `A0` in `adc_read.ino` into the corresponding analog pin number (`A0-A6` for `P6.0-6.6`, `A12` for `P7.0`).
9. Connect the ground of the Launchpad to the ground of your circuit.
10. Press the reset button on your Launchpad.
11. In the command line on your computer, navigate into the `adc_read` folder and run `python adc_read.py -D COM###` (substitute ### with the proper number for your Launchpad from Device Manager).
    - Note: If you are running this on a Mac, open the adc_read.py file and change the line `ani = anim.FuncAnimation(fig, update, fargs=(l1,ser, line, data, num_read), interval=0.001*num_read)` to `ani = anim.FuncAnimation(fig, update, fargs=(l1,ser, line, data, num_read), interval=num_read)`.
12. If you see a graph on your screen with a signal that changes when you make noises at your microphone and is centered in the middle of the graph, that means your Launchpad is able to read the output of your mic board circuitry properly and output it to your computer. You have succeeded!


# <span style="color:green">CHECKOFF</span>

- Have all questions, code, and plots completed in this notebook.
- Demonstrate your car's preprogrammed path, which **includes all 4 commands (drive straight far, turn left, drive straight close, and turn right).**
- Demonstrate that your **Launchpad is reading the mic board circuitry output correctly.**
- **Be prepared to answer conceptual questions about the lab.**
 
### Checkoff Request Link: https://links.eecs16b.org/lab-checkoff-sp22

## SAVE ALL YOUR DATA!!

- **Data stored on the lab computers often gets deleted automatically.** Please store it on your personal flash drive or cloud storage like Google Drive, and not on the lab computers! If you used DataHub, the notebook should save through your CalNet ID. **<span style='color:red'> Make sure to back up your .ino files. You will need them for Integration/Final Demo!</span>**
- **You will need to report your design processes in the final lab report.**